In [0]:
import pandas as pd
from pyspark.sql.functions import col, lit, when, expr

# --- 1. PREPARAR BASE DE HISTÓRICO (Apenas Clientes Ativos) ---
# Lemos a Silver para ter os dados descritivos (InternetService = 'Fiber optic')
df_hist_silver = spark.table("workspace.churn_zero.history_silver")

# Filtramos apenas quem NÃO cancelou (Churn = 0 ou 'No')
# Assumindo que na Silver o Churn é 'No' (string) ou 0 (int) dependendo da sua limpeza. 
# Ajuste o filtro abaixo se necessário.
df_hist_active = df_hist_silver.filter(col("Churn") == 0).withColumn("origin", lit("history_active"))

# Para clientes históricos ativos, assumimos uma probabilidade de churn baixa (ou 0) 
# pois não rodamos o modelo neles agora, ou usamos o score se tiver. 
# Vamos definir 0.0 para focar o risco na base de inferência, mas manter a oportunidade de venda.
df_hist_active = df_hist_active.withColumn("churn_probability", lit(0.0))


# --- 2. PREPARAR BASE DE INFERÊNCIA (Novos Dados com Previsão) ---
# Lemos a tabela final de inferência que tem a probabilidade
df_inf_pred = spark.table("workspace.churn_zero.inference_churn").select("customerID", "churn_probability", "prediction")

# Lemos a Silver da inferência para ter os textos
df_inf_silver = spark.table("workspace.churn_zero.inference_silver")

# Fazemos o Join para ter: ID + Probabilidade + Dados Descritivos
df_inf_full = df_inf_silver.join(df_inf_pred, "customerID", "inner").withColumn("origin", lit("inference"))


# --- 3. UNIFICAR AS BASES ---
# Selecionamos as colunas comuns importantes para o Vendedor e para a IA
common_cols = [
    "customerID", "gender", "SeniorCitizen", "Partner", "Dependents", 
    "tenure", "PhoneService", "MultipleLines", "InternetService", 
    "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", 
    "StreamingTV", "StreamingMovies", "Contract", "PaperlessBilling", 
    "PaymentMethod", "MonthlyCharges", "TotalCharges", 
    "feedback_topic", "churn_probability", "origin"
]

# Garantir que ambas tenham as mesmas colunas
df_final_base = df_hist_active.select(common_cols).unionByName(df_inf_full.select(common_cols))

# Criamos uma View Temporária para usar SQL (ai_gen) na próxima célula
df_final_base.createOrReplaceTempView("all_customers_view")

print(f"Base unificada criada com {df_final_base.count()} clientes (Histórico Ativo + Inferência).")

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW customers_enriched AS
SELECT 
    *,
    -- Contagem de Add-ons (Importante para a regra de fidelização)
    (CASE WHEN OnlineSecurity != 'No' THEN 1 ELSE 0 END + 
     CASE WHEN OnlineBackup != 'No' THEN 1 ELSE 0 END + 
     CASE WHEN DeviceProtection != 'No' THEN 1 ELSE 0 END + 
     CASE WHEN TechSupport != 'No' THEN 1 ELSE 0 END +
     CASE WHEN StreamingTV != 'No' THEN 1 ELSE 0 END +
     CASE WHEN StreamingMovies != 'No' THEN 1 ELSE 0 END) as num_addons,
     
    -- Classificação de Risco para o Vendedor (Visual)
    CASE 
        WHEN churn_probability >= 0.7 THEN 'CRÍTICO'
        WHEN churn_probability >= 0.5 THEN 'ALTO'
        WHEN feedback_topic = 'sales_opportunity' THEN 'OPORTUNIDADE'
        ELSE 'NORMAL'
    END as status_venda
FROM all_customers_view;

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.churn_zero.app_layer_final AS
SELECT 
    *,
    
    -- O CÉREBRO: Gera a instrução tática para o vendedor
    ai_gen(
        'Atue como um Estrategista de Vendas Sênior da Terra Signal. 
        Gere uma **Instrução Interna Curta** (máximo 25 palavras) para o vendedor que vai ligar para este cliente.
        Seja diretivo: "O cliente tem X, ofereça Y".

        DADOS DO CLIENTE:
        - Risco de Churn (Probabilidade): ' || round(churn_probability * 100, 1) || '%
        - Motivo/Feedback: ' || feedback_topic || '
        - Tecnologia: ' || InternetService || '
        - Valor Mensal: $' || MonthlyCharges || '
        - Contrato: ' || Contract || '
        - Qtd Add-ons: ' || num_addons || '

        RACIOCÍNIO ESTRATÉGICO:
        1. SE RISCO ALTO (>50%) ou FEEDBACK NEGATIVO: Foco em Retenção.
           - Preço alto/Concorrência na Fibra -> Autorize desconto de 15% por fidelidade 1 ano.
           - Problema Técnico (DSL/Fibra) -> Ofereça visita técnica VIP ou troca de equipamento.
           - "lost_sales_opportunity" -> Win-back: ofereça exatamente o que ele queria.
        
        2. SE "sales_opportunity" (Sem Internet): 
           - Foco total em Venda Nova. Ofereça pacote Fibra com instalação grátis.
        
        3. SE RISCO BAIXO (Cliente Seguro): Foco em Blindagem/Upsell.
           - Poucos Add-ons -> Ofereça TechSupport grátis por 3 meses (degustação).
           - DSL Satisfeito -> Ofereça upgrade para Fibra com desconto.
           - Fibra Satisfeito -> Agradeça a fidelidade (não mexa em time que está ganhando).

        SAÍDA:
        Apenas a frase da ação. Ex: "Cliente Fibra com risco de preço. Ofereça 15% de desconto e fidelidade."'
    ) as recommended_action

FROM customers_enriched;

In [0]:
# Carrega a tabela com a IA gerada
df_app = spark.table("workspace.churn_zero.app_layer_final").toPandas()

# --- CÁLCULO DO PRIORITY SCORE (Para ordenar a lista no site) ---
# Lógica: (Probabilidade Churn * Valor) + Bônus de Oportunidade
# Queremos ligar primeiro para: 
# 1. Quem vai sair e paga muito.
# 2. Quem quer comprar (sales_opportunity).

def calculate_priority(row):
    base_score = row['churn_probability'] * row['MonthlyCharges']
    
    # Bônus: Se é uma oportunidade de venda clara, joga pro topo (equivale a um risco alto)
    if row['feedback_topic'] == 'sales_opportunity':
        base_score += 100 # Bônus alto para vender logo
    
    # Bônus: Se é risco crítico de fibra (valor alto), garante prioridade
    if row['status_venda'] == 'CRÍTICO' and row['InternetService'] == 'Fiber optic':
        base_score *= 1.5
        
    return round(base_score, 2)

df_app['priority_score'] = df_app.apply(calculate_priority, axis=1)

# Ordenar para o site (Do maior score para o menor)
df_app = df_app.sort_values(by='priority_score', ascending=False)

# Formatar Probabilidade para % bonita
df_app['churn_probability_display'] = (df_app['churn_probability'] * 100).round(1).astype(str) + '%'


In [0]:
%sql
SELECT
  *
FROM
  workspace.churn_zero.site_feed_table;

In [0]:
# Mapeamento dos valores de feedback_topic para português sem underlines
feedback_translation = {
    "competitor_offer": "Oferta do Concorrente",
    "high_price": "Alto Preço",
    "sales_opportunity": "Oportunidade de venda",
    "lost_sales_opportunity": "Oportunidade de venda perdida",
    "personal_reasons": "Questões pessoais",
    "customer_service": "Atendimento ao Cliente",
    "billing_issues": "Problemas de pagamento",
    "general_satisfaction": "Satisfeito com os serviços",
    "internet_connection": "Problema de conexão",
    "contract_terms": "Questões contratuais",
    "other": "Outro"
}
service_translation = {
    "DSL": "DSl",
    "Fiber optic": "Fibra ótica",
    "No": "Sem serviço de internet"
}
payment_translation ={
    "Electronic check": "Fatura digital",
    "Mailed check": "Fatura física",
    "Bank transfer (automatic)": "Débito automático",
    "Credit card (automatic)": "Cartão de crédito"
}
contract_translation = {
    "Month-to-month": "Mensal",
    "One year": "1 ano",
    "Two year": "2 anos"
}
df_app['feedback_topic'] = df_app['feedback_topic'].map(feedback_translation).fillna(df_app['feedback_topic'].str.replace('_', ' ').str.title())
df_app['Contract'] = df_app['Contract'].map(contract_translation).fillna(df_app['Contract'].str.replace('_', ' ').str.title())
df_app['InternetService'] = df_app['InternetService'].map(service_translation).fillna(df_app['InternetService'].str.replace('_', ' ').str.title())
df_app['PaymentMethod'] = df_app['PaymentMethod'].map(payment_translation).fillna(df_app['PaymentMethod'].str.replace('_', ' ').str.title())

df_app.to_csv("/Workspace/Users/joaovab@al.insper.edu.br/Hackathon_Terra_Signal_grupo_1/site_data.csv", index=False)

# Salva no Unity Catalog como tabela final de consumo
spark_final = spark.createDataFrame(df_app)
spark_final.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("workspace.churn_zero.site_feed_table")

print("✅ Arquivo 'site_data.csv' gerado e tabela 'site_feed_table' atualizada!")
print("Squad B pode usar esses dados no Streamlit agora.")
display(df_app.head(5))

In [0]:
spark_final.printSchema()